In [ ]:
pip install confluent-kafka

In [ ]:
import json
import logging
import requests
import time
from confluent_kafka import Producer

# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')
logger = logging.getLogger("ConfluentKafkaProducer")

producer_config = {
    'bootstrap.servers': 'kafka-1:9092,kafka-2:9093,kafka-3:9094',
    'client.id': 'flight-producer-client',
    'acks': 'all',  
    'batch.size': 16384,  
    'linger.ms': 5,  
    'compression.codec': 'snappy', 
    'max.in.flight': 5 
}
producer = Producer(producer_config)

api_url = "http://host.docker.internal:5000/api/flights"
topic = "flight-producer"

def fetch_data():
    try:
        response = requests.get(api_url, timeout=10)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        logger.error(f"API request failed: {e}")
        return None

def delivery_report(err, msg):
    if err is not None:
        logger.error(f"Delivery failed: {err}")
    else:
        logger.info(f"Message delivered to {msg.topic()} [{msg.partition()}]")

def send_to_kafka(data):
    try:
        producer.produce(
            topic,
            value=json.dumps(data).encode("utf-8"),
            callback=delivery_report
        )
        producer.flush()
    except Exception as e:
        logger.error(f"Failed to send message to Kafka: {e}")

def process_flights(data):
    for flight in data.get("flights", []):
        try:
            dep = flight["departure_city"]
            arr = flight["arrival_city"]
            logger.info(f"Flight from {dep} to {arr}")
        except Exception as e:
            logger.warning(f"Failed to process flight: {e}")

def main():
    logger.info("Starting continuous Kafka producer...")

    while True:
        data = fetch_data()
        if data:
            process_flights(data)
            send_to_kafka(data)
        else:
            logger.warning("No data fetched from API.")
        
        time.sleep(10)

if __name__ == "__main__":
    main()
